In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords

In [2]:
import os

data=pd.read_csv('/kaggle/input/zomato-bangalore-restaurants/zomato.csv')

In [3]:
data.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [4]:
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))

def clean_text(review):
    
    rating_match = re.search(r'Rated ([\d.]+)', review)
    if rating_match:
        rating = float(rating_match.group(1))
        review_text = review.replace(rating_match.group(0), '').strip()
    else:
        return '', 0  

    
    review_text = re.sub(r'http\S+', '', review_text)  
    review_text = re.sub(r'[^a-zA-Z\s]', '', review_text)  
    review_text = review_text.lower()  
    review_text = review_text.split()  
    review_text = [word for word in review_text if word not in STOPWORDS] 

    return ' '.join(review_text), rating


data['cleaned_reviews'], data['rating'] = zip(*data['reviews_list'].apply(clean_text))


data['sentiment'] = data['rating'].apply(lambda x: 1 if x >= 4 else 0)


print(data[['cleaned_reviews', 'sentiment']].head())



[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
                                     cleaned_reviews  sentiment
0  ratedn beautiful place dine inthe interiors ta...          1
1  ratedn dinner family turned good choose suitab...          1
2  ratedn ambience good enough pocket friendly ca...          0
3  ratedn great food proper karnataka style full ...          1
4  ratedn good restaurant neighbourhood buffet sy...          1


In [5]:
data.shape

(51717, 20)

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
max_vocab_size=5000
max_sequence_length=100
reviews = data['cleaned_reviews'].astype(str)  
labels = data['sentiment']
tokenizer = Tokenizer(num_words=max_vocab_size)
tokenizer.fit_on_texts(reviews)

X = tokenizer.texts_to_sequences(reviews)
X = pad_sequences(X, maxlen=max_sequence_length)

y = data['sentiment'].values

In [8]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout


model = Sequential()


model.add(Embedding(input_dim=max_vocab_size,output_dim=128))


model.add(LSTM(units=64, return_sequences=False))


model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))


model.add(Dense(1, activation='sigmoid'))
model.build(input_shape=(None, max_sequence_length))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 128)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 693,633 (2.65 MB)

 Trainable params: 693,633 (2.65 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:

history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)


Epoch 1/5
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 47s 43ms/step - accuracy: 0.7189 - loss: 0.5272 - val_accuracy: 0.8286 - val_loss: 0.3775
Epoch 2/5
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 44s 43ms/step - accuracy: 0.8837 - loss: 0.2824 - val_accuracy: 0.8918 - val_loss: 0.2996
Epoch 3/5
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 45s 43ms/step - accuracy: 0.9426 - loss: 0.1568 - val_accuracy: 0.8982 - val_loss: 0.2660
Epoch 4/5
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 44s 43ms/step - accuracy: 0.9642 - loss: 0.1034 - val_accuracy: 0.9256 - val_loss: 0.2369
Epoch 5/5
1035/1035 ━━━━━━━━━━━━━━━━━━━━ 44s 43ms/step - accuracy: 0.9784 - loss: 0.0649 - val_accuracy: 0.9283 - val_loss: 0.2476


In [11]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")

324/324 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.9251 - loss: 0.2446
Test Accuracy: 0.93


In [12]:
import pickle

# Save the tokenizer
with open('tokenizer.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
with open('tokenizer.pkl', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [14]:
new_sentences = [
    "The food was amazing and the service was great!",
    "I had a terrible experience at the restaurant.",
    "I recently visited The Cozy Bistro for dinner, and while the experience had its highlights, there were some areas that could use improvement.Positive Aspects: The ambiance of The Cozy Bistro is absolutely charming. The warm lighting and rustic decor create a welcoming and relaxed atmosphere, perfect for a casual dinner. The service staff were friendly and attentive, which made the evening pleasant. The highlight of the meal was definitely the mushroom risotto—creamy, flavorful, and cooked to perfection. The dessert, a decadent chocolate lava cake, was also a treat.Areas for Improvement: However, there were a few downsides. The wait time for our food was significantly longer than expected, even though the restaurant wasn’t particularly busy. When the food arrived, it was lukewarm rather than hot, which was disappointing. Additionally, the prices felt a bit steep for the portion sizes. While I understand that quality often comes at a cost, it felt like the value wasn’t quite there.Overall, The Cozy Bistro has potential with its inviting atmosphere and tasty dishes. If they can improve on the service speed and ensure the food is served at the right temperature, it could be a top choice for a delightful meal out."
]


new_sequences = tokenizer.texts_to_sequences(new_sentences)


max_sequence_length = 100  
new_padded_sequences = pad_sequences(new_sequences, maxlen=max_sequence_length)


In [15]:

predictions = model.predict(new_padded_sequences)


sentiment_labels = ['Positive' if pred > 0.5 else 'Negative' for pred in predictions]

for sentence, sentiment in zip(new_sentences, sentiment_labels):
    print(f"Sentence: '{sentence}' -> Sentiment: {sentiment}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
Sentence: 'The food was amazing and the service was great!' -> Sentiment: Positive
Sentence: 'I had a terrible experience at the restaurant.' -> Sentiment: Negative
Sentence: 'I recently visited The Cozy Bistro for dinner, and while the experience had its highlights, there were some areas that could use improvement.Positive Aspects: The ambiance of The Cozy Bistro is absolutely charming. The warm lighting and rustic decor create a welcoming and relaxed atmosphere, perfect for a casual dinner. The service staff were friendly and attentive, which made the evening pleasant. The highlight of the meal was definitely the mushroom risotto—creamy, flavorful, and cooked to perfection. The dessert, a decadent chocolate lava cake, was also a treat.Areas for Improvement: However, there were a few downsides. The wait time for our food was significantly longer than expected, even though the restaurant wasn’t particularly busy. When the food arrived, it was luke

In [16]:
dataset_path='/kaggle/input/zomato-bangalore-restaurants/zomato.csv'
from geopy.distance import great_circle
class Recommendation_hotel():
    def __init__(self,dataset_path):
        self.dataset_path=dataset_path
        self.hotel_data=pd.read_csv(dataset_path)
        self.hotel_data=self.hotel_data['address',	'name',	'online_order',	'book_table',	'rate',	'votes',	'phone',	'location',	'rest_type',	'dish_liked',	'cuisines',	'approx_cost(for two people)',	'reviews_list',	'menu_item',	'listed_in(type)',	'listed_in(city)']
        
        
    def get_location_data

SyntaxError: invalid syntax (4035226280.py, line 8)